In [8]:
import lmql
import nest_asyncio
import numpy as np
from openai import OpenAI

client = OpenAI(
  organization='org-oFzgDwrVnbgWoVIjiAzwwMaj',
)

In [77]:
#Define text description we want scenic code for
i = 7
text_description_path = f"examples/carla_text/carlaChallenge{i}.scenic"
text_description = open(text_description_path, 'r').read()

In [3]:
#Define example scenic program
j = 6
scenic_example_path = f"examples/carla_scenic/carlaChallenge{j}.scenic"
scenic_program = open(scenic_example_path, 'r').read()

In [4]:
#Define scenic code building template
scenic_template_path = f"src/constraints/lmql_template.scenic"
scenic_template = open(scenic_template_path, 'r').read()

In [9]:
#Define any known variable sets
towns = list(np.load('src/constraints/blueprints/towns.npy'))
vehicles = list(np.load('src/constraints/blueprints/vehicles.npy'))

In [67]:
nest_asyncio.apply()
@lmql.query()
def generate_text_description(text_description, scenic_program):
    '''lmql
    "This is an example of a Scenic program: {scenic_program}\n"
    
    "We will write a new Scenic program based on the inputted natural language description."
    "The text description is: {text_description}\n"
    
    "This is the template we will fill out to write our Scenic program: {scenic_template}."
    "We will fill out each section marked with TODO.\n"
    
    "The first part of the new Scenic Program is a text description of the scenario."
    "This will complete the TEXT_DESCRIPTION_TODO section in the given template."
    "Please repeat the all of the text description in this part as a multi-line comment [TEXT_DESCRIPTION].\n" where type(TEXT_DESCRIPTION) == str and STOPS_BEFORE(TEXT_DESCRIPTION, "##")
    
    "The next section of the Scenic Program is setting the map and model."
    "This gives us all definitions of objects and their properties such as vehicle types, pedestrians, and road library."
    "Please select a CARLA map [CARLA_MAP_NAME] that will complete the CARLA_MAP_NAME_TODO in the template.\n" where type(CARLA_MAP_NAME) == str and CARLA_MAP_NAME in towns
    
    "The next section of our template is CONSTANTS."
    "First pick the vehicle blueprint ID that most closely matches the description but is supported in CARLA: [EGO_VEHICLE_BLUEPRINT_ID]" where type(EGO_VEHICLE_BLUEPRINT_ID) == str and EGO_VEHICLE_BLUEPRINT_ID in vehicles
    "The ego vehicle should also have a speed [EGO_VEHICLE_SPEED] based on the text description.\n" where INT(EGO_VEHICLE_SPEED) 
    
    "The next section of our template is OTHER CONSTANTS."
    "Include any other variables that you may need for the rest of the scenic program especially those "
    "that may describe behaviors: [OTHER_VARIABLES].\n" where STOPS_BEFORE(OTHER_VARIABLES, "\n\n")
    
    "The next section of the template is DEFINING BEHAVIORS."
    "Define the behavior of the ego vehicle and any other vehicles according to the description."
    "This where the dynamics of a scene occur [VEHICLE_BEHAVIORS]\n" where STOPS_BEFORE(VEHICLE_BEHAVIORS, "DEFINING SPATIAL RELATIONS")

    "The next section of the template is DEFINING SPATIAL RELATIONS."
    "Define the spatial relation of the ego vehicle and any other vehicles or obstacles"
    "according to the description: [SPATIAL_RELATIONS]\n" where STOPS_BEFORE(SPATIAL_RELATIONS, "require")

    "The last section is POSTCONDITIONS."
    "Summarize any postconditions or properties that the text description may have implied."
    "[POSTCONDITIONS]\n"
    
    
    return {
        "TEXT_DESCRIPTION": TEXT_DESCRIPTION,
        "CARLA_MAP_NAME" : CARLA_MAP_NAME,
        "EGO_VEHICLE_BLUEPRINT_ID" : EGO_VEHICLE_BLUEPRINT_ID,
        "EGO_VEHICLE_SPEED" : EGO_VEHICLE_SPEED,
        "OTHER_VARIABLES" : OTHER_VARIABLES,
        "VEHICLE_BEHAVIORS" : VEHICLE_BEHAVIORS,
        "SPATIAL_RELATIONS" : SPATIAL_RELATIONS,
        "POSTCONDITIONS" : POSTCONDITIONS
    }
    
    
    '''

In [78]:
ans = generate_text_description(text_description, scenic_program)

print(ans["TEXT_DESCRIPTION"])



"""
Scenario Description
Based on 2019 Carla Challenge Traffic Scenario 07.
Ego-vehicle is going straight at an intersection but a crossing vehicle 
runs a red light, forcing the ego-vehicle to perform a collision avoidance maneuver.
Note: The traffic light control is not implemented yet, but it will soon be. 
"""




In [79]:
print(ans["CARLA_MAP_NAME"])

Town01


In [80]:
print(ans['EGO_VEHICLE_BLUEPRINT_ID'])

vehicle.tesla.model3


In [81]:
print(ans['OTHER_VARIABLES'])


CROSSING_VEHICLE_SPEED = 15
DIST_THRESHOLD = 10
BREAK_INTENSITY = 0.8
DIST_TO_INTERSECTION = 20
DIST_TO_CROSSING_VEHICLE = 15
YIELD_THRESHOLD = 5
TURNING_ANGLE = 45 deg
TURNING_SPEED = 5


In [82]:
print(ans["SPATIAL_RELATIONS"])


#GEOMETRY
#Find lanes that have a lane to their left in the opposite direction
laneSecsWithLeftLane = []
for lane in network.lanes:
    for laneSec in lane.sections:
        if laneSec._laneToLeft is not None:
            if laneSec._laneToLeft.isForward is not laneSec.isForward:
                laneSecsWithLeftLane.append(laneSec)

assert len(laneSecsWithLeftLane) > 0, \
    'No lane sections with adjacent left lane with opposing \
    traffic direction in network.'

initLaneSec = Uniform(*laneSecsWithLeftLane)
leftLaneSec = initLaneSec._laneToLeft

spawnPt = new OrientedPoint on initLaneSec.centerline

#PLACEMENT
crossingVehicle = new Car on leftLaneSec.centerline,
    with behavior CrossingVehicleBehavior()

ego = new Car at spawnPt,
    with behavior EgoBehavior(leftLaneSec)
    
#Make sure the crossing vehicle is at a visible section of the lane



In [83]:
print(ans["POSTCONDITIONS"])

 
require crossingVehicle can see ego
require (distance from crossingVehicle to ego) < DIST_TO_CROSSING_VEHICLE
require (distance from crossingVehicle to intersection) > DIST_TO_INTERSECTION
